In [2]:
import pandas as pd
import requests

In [3]:
df = requests.get('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2016-03.parquet')
df = df.read_parquet('data\yellow_tripdata_2023-01.parquet')


In [4]:
df.to_csv('data/uber_data.csv')

In [6]:
df = pd.read_csv('data/uber_data.csv',low_memory=False)

In [7]:
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

In [8]:
df = df.drop_duplicates().reset_index(drop=True)
df['trip_id'] = df.index

In [26]:
df.columns

Index(['Unnamed: 0', 'VendorID', 'tpep_pickup_datetime',
       'tpep_dropoff_datetime', 'passenger_count', 'trip_distance',
       'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID',
       'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount',
       'tolls_amount', 'improvement_surcharge', 'total_amount',
       'congestion_surcharge', 'airport_fee', 'trip_id'],
      dtype='object')

In [12]:
datetime_dim = df[['tpep_pickup_datetime','tpep_dropoff_datetime']].reset_index(drop=True)
datetime_dim['tpep_pickup_datetime'] = datetime_dim['tpep_pickup_datetime']
datetime_dim['pick_hour'] = datetime_dim['tpep_pickup_datetime'].dt.hour
datetime_dim['pick_day'] = datetime_dim['tpep_pickup_datetime'].dt.day
datetime_dim['pick_month'] = datetime_dim['tpep_pickup_datetime'].dt.month
datetime_dim['pick_year'] = datetime_dim['tpep_pickup_datetime'].dt.year
datetime_dim['pick_weekday'] = datetime_dim['tpep_pickup_datetime'].dt.weekday

datetime_dim['tpep_dropoff_datetime'] = datetime_dim['tpep_dropoff_datetime']
datetime_dim['drop_hour'] = datetime_dim['tpep_dropoff_datetime'].dt.hour
datetime_dim['drop_day'] = datetime_dim['tpep_dropoff_datetime'].dt.day
datetime_dim['drop_month'] = datetime_dim['tpep_dropoff_datetime'].dt.month
datetime_dim['drop_year'] = datetime_dim['tpep_dropoff_datetime'].dt.year
datetime_dim['drop_weekday'] = datetime_dim['tpep_dropoff_datetime'].dt.weekday


datetime_dim['datetime_id'] = datetime_dim.index
datetime_dim = datetime_dim[['datetime_id', 'tpep_pickup_datetime', 'pick_hour', 'pick_day', 'pick_month', 'pick_year', 'pick_weekday',
                             'tpep_dropoff_datetime', 'drop_hour', 'drop_day', 'drop_month', 'drop_year', 'drop_weekday']]
#
datetime_dim.head()

,datetime_id,tpep_pickup_datetime,pick_hour,pick_day,pick_month,pick_year,pick_weekday,tpep_dropoff_datetime,drop_hour,drop_day,drop_month,drop_year,drop_weekday
0,0,2023-01-01 00:32:10,0,1,1,2023,6,2023-01-01 00:40:36,0,1,1,2023,6
1,1,2023-01-01 00:55:08,0,1,1,2023,6,2023-01-01 01:01:27,1,1,1,2023,6
2,2,2023-01-01 00:25:04,0,1,1,2023,6,2023-01-01 00:37:49,0,1,1,2023,6
3,3,2023-01-01 00:03:48,0,1,1,2023,6,2023-01-01 00:13:25,0,1,1,2023,6
4,4,2023-01-01 00:10:29,0,1,1,2023,6,2023-01-01 00:21:19,0,1,1,2023,6


In [27]:
rate_code_type = {
    1:"Standard rate",
    2:"JFK",
    3:"Newark",
    4:"Nassau or Westchester",
    5:"Negotiated fare",
    6:"Group ride"
}

rate_code_dim = df[['RatecodeID']].drop_duplicates().reset_index(drop=True)
rate_code_dim['rate_code_id'] = rate_code_dim.index
rate_code_dim['rate_code_name'] = rate_code_dim['RatecodeID'].map(rate_code_type)
rate_code_dim = rate_code_dim[['rate_code_id','rate_code_name']]


In [28]:
rate_code_dim.head()

,rate_code_id,rate_code_name
0,0,Standard rate
1,1,JFK
2,2,Nassau or Westchester
3,3,NaN
4,4,Negotiated fare


In [23]:
payment_type_name = {
    1:"Credit card",
    2:"Cash",
    3:"No charge",
    4:"Dispute",
    5:"Unknown",
    6:"Voided trip"
}
payment_type_dim = df[['payment_type']].drop_duplicates().reset_index(drop=True)
payment_type_dim['payment_type_id'] = payment_type_dim.index
payment_type_dim['payment_type_name'] = payment_type_dim['payment_type'].map(payment_type_name)
payment_type_dim = payment_type_dim[['payment_type_id','payment_type_name']]

In [24]:
payment_type_dim.head()

,payment_type_id,payment_type_name
0,0,Cash
1,1,Credit card
2,2,Dispute
3,3,No charge
4,4,NaN


In [30]:

pickup_location_dim = df[['pickup_longitude', 'pickup_latitude']].reset_index(drop=True)
pickup_location_dim['pickup_location_id'] = pickup_location_dim.index
pickup_location_dim = pickup_location_dim[['pickup_location_id','pickup_latitude','pickup_longitude']] 


dropoff_location_dim = df[['dropoff_longitude', 'dropoff_latitude']].reset_index(drop=True)
dropoff_location_dim['dropoff_location_id'] = dropoff_location_dim.index
dropoff_location_dim = dropoff_location_dim[['dropoff_location_id','dropoff_latitude','dropoff_longitude']]


fact_table = df.merge(datetime_dim, left_on='trip_id', right_on='datetime_id') \
            .merge(payment_type_dim, left_on='payment_type', right_on='payment_type_id') \
            .merge(rate_code_dim, left_on='RatecodeID', right_on='rate_code_id') \
            .merge(pickup_location_dim, left_on='trip_id', right_on='pickup_location_id') \
            .merge(dropoff_location_dim, left_on='trip_id', right_on='dropoff_location_id')\
            [['VendorID', 'datetime_id', 'passenger_count',
            'trip_distance', 'rate_code_id', 'store_and_fwd_flag', 'pickup_location_id', 'dropoff_location_id',
            'payment_type_id', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
            'improvement_surcharge', 'total_amount']]

In [31]:
fact_table.columns

Index(['trip_id', 'VendorID', 'datetime_id', 'passenger_count',
       'trip_distance', 'rate_code_id', 'store_and_fwd_flag', 'PULocationID',
       'DOLocationID', 'payment_type_id', 'fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount'],
      dtype='object')